# Load in the data - X-Fact

In [1]:
import pandas as pd
import os
os.chdir(r'C:\Users\annal\OneDrive\Desktop\llm-misinfo-eval\data')  #this line is what I've been using to solve a no such directory error - copy paste the path of data folder
df = pd.read_csv(r"x_fact_dataset/x-fact-including-en/zeroshot.tsv", delimiter="\t", on_bad_lines="skip")
#TODO read in right csv!

# Get results from Gemini

In [8]:
def return_prompt_1(STATEMENT):
    return f"Rate the truthfulness of the following statement: “{STATEMENT}” Provide a score from 0 to 100, where 0 represents definitively false and 100 represents definitively true. Do not provide any explanations, only respond with the numerical score."

In [20]:
import vertexai
from vertexai.language_models import TextGenerationModel
from vertexai.preview.generative_models import GenerativeModel

In [68]:
def call_model(prompt):
    vertexai.init(project='llm-misinfo-eval', location='us-central1')
    parameters = {
        # 'temperature':0,
        # 'max_output_tokens':256,
        # 'top_p':0.8,
        # 'top_k':40
    }
    model = GenerativeModel('gemini-1.0-pro')
    resp = model.generate_content(prompt, **parameters)
    return resp

def get_text_resp(model_resp):
    return model_resp.candidates[0].content.parts[0].text
# resp = get_text_resp(call_model('what is your name?'))

In [86]:
from tqdm import trange
import time
df_results_prompt_1_list = []
for i in trange(300):
    tmp = {'modelResp':None, 'trueLabel':df['label'][i]}
    claim = df['claim'][i]
    prompt = return_prompt_1(claim)
    model_resp = call_model(prompt)
    model_text_resp = get_text_resp(model_resp)
    try:
        tmp['modelResp'] = float(model_text_resp)
    except:
        tmp['modelResp'] = float(model_text_resp)
    df_results_prompt_1_list.append(tmp)
    time.sleep(10)

 24%|██████████████████▉                                                            | 72/300 [21:31<1:08:11, 17.94s/it]


InternalServerError: 500 Internal error encountered.

# Saving Gemini's Responses to CSV

In [94]:
df_results_prompt_1 = pd.DataFrame(df_results_prompt_1_list)
df_results_prompt_1.to_csv("gemini_results", sep=',', index=False, encoding='utf-8')

(72, 2)
